In [1]:
import pandas as pd
import numpy as np
import warnings
import sys, os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append(os.path.abspath('..'))
from core.DICT import TYPE2NUM

label_path = '../../data/labels/full_non_transit_stopped_radar_labels.csv'
image_path = 'nontransit_track_images/'

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

import warnings
warnings.filterwarnings("ignore", message="Failed to load image Python extension")

Using device: mps


#### CNNBaseline

In [9]:
BATCH_SIZE = 32
NUM_HIDDEN_1 = 32
NUM_HIDDEN_2 = 64
NUM_HIDDEN_3 = 128
IN_CHANNELS = 3

NUM_CLASSES = len(TYPE2NUM)

from dataloader_imgs import get_type_datasets
train_loader, val_loader, test_loader = get_type_datasets(label_path, image_path, batch_size = BATCH_SIZE)

In [10]:
import torch

all_labels = []

for _, labels in test_loader:
    all_labels.append(labels)

all_labels = torch.cat(all_labels)

# Convert to numpy array
labels_np = all_labels.numpy()

# Get unique labels and their counts
unique_labels, counts = np.unique(labels_np, return_counts=True)

for label, count in zip(unique_labels, counts):
    print(f"Label {label}: {count} samples")


Label 0: 62 samples
Label 1: 3 samples
Label 2: 21 samples
Label 3: 11 samples
Label 4: 2 samples
Label 5: 40 samples
Label 6: 29 samples


In [6]:
from models import CNNBaseline
from engine import train_model, save_model

cnn_baseline_1 = CNNBaseline(IN_CHANNELS, NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)
model_name = "nontransit_cnn_baseline_best_10"

train_config = {
    "model": cnn_baseline_1,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optimizer,
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

#train and save model
TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss = train_model(**train_config)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)


KeyboardInterrupt: 

In [5]:
#Retrain for further epochs
cnn_baseline_1 = CNNBaseline(IN_CHANNELS, NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)

checkpoint_path = f"models/{model_name}.pth"
cnn_baseline_1.load_state_dict(torch.load(checkpoint_path))
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)
model_name = "nontransit_cnn_baseline_best_20"

train_config_resume = {
    "model": cnn_baseline_1,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

# 5. Resume training
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train_model(**train_config_resume)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)


Epoch 1/10
Train Loss: 1.2608 | Train Acc: 0.5124
Val   Loss: 1.3581 | Val   Acc: 0.4412


Epoch 2/10
Train Loss: 1.1791 | Train Acc: 0.5424
Val   Loss: 1.3505 | Val   Acc: 0.4882


Epoch 3/10
Train Loss: 1.1375 | Train Acc: 0.5621
Val   Loss: 1.3659 | Val   Acc: 0.4588


Epoch 4/10
Train Loss: 1.0781 | Train Acc: 0.5811
Val   Loss: 1.4219 | Val   Acc: 0.4235


Epoch 5/10
Train Loss: 1.0376 | Train Acc: 0.6060
Val   Loss: 1.4253 | Val   Acc: 0.4412


Epoch 6/10
Train Loss: 0.9729 | Train Acc: 0.6221
Val   Loss: 1.4652 | Val   Acc: 0.4412


Epoch 7/10
Train Loss: 0.8995 | Train Acc: 0.6623
Val   Loss: 1.4705 | Val   Acc: 0.4647


Epoch 8/10
Train Loss: 0.8575 | Train Acc: 0.6835
Val   Loss: 1.5046 | Val   Acc: 0.4471


Epoch 9/10
Train Loss: 0.7905 | Train Acc: 0.7032
Val   Loss: 1.5718 | Val   Acc: 0.4765


Epoch 10/10
Train Loss: 0.6831 | Train Acc: 0.7624
Val   Loss: 1.7150 | Val   Acc: 0.4647


### Resnet

In [9]:
from models import get_resnet50_classifier

resnet50 = get_resnet50_classifier(NUM_CLASSES).to(device)
optimizer = optim.Adam(resnet50.parameters(), lr=1e-3)
model_name = "nontransit_resnet50_best_10"

train_config = {
    "model": resnet50,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optimizer, 
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

#train loop
TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss = train_model(**train_config)
save_model(TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_val_loss, best_model_wts, model_name)

Epoch 1/10
Train Loss: 1.6398 | Train Acc: 0.3432
Val   Loss: 1.5954 | Val   Acc: 0.3333


Epoch 2/10
Train Loss: 1.4645 | Train Acc: 0.4497
Val   Loss: 1.5222 | Val   Acc: 0.3929


Epoch 3/10
Train Loss: 1.3784 | Train Acc: 0.4852
Val   Loss: 1.4431 | Val   Acc: 0.4286


Epoch 4/10
Train Loss: 1.3179 | Train Acc: 0.5274
Val   Loss: 1.4107 | Val   Acc: 0.4405


Epoch 5/10
Train Loss: 1.2922 | Train Acc: 0.5163
Val   Loss: 1.3988 | Val   Acc: 0.4583


Epoch 6/10
Train Loss: 1.2393 | Train Acc: 0.5481
Val   Loss: 1.3706 | Val   Acc: 0.4881


Epoch 7/10
Train Loss: 1.2049 | Train Acc: 0.5651
Val   Loss: 1.3600 | Val   Acc: 0.4821


Epoch 8/10
Train Loss: 1.1769 | Train Acc: 0.5865
Val   Loss: 1.3534 | Val   Acc: 0.4702


Epoch 9/10
Train Loss: 1.1603 | Train Acc: 0.5754
Val   Loss: 1.3308 | Val   Acc: 0.4702


Epoch 10/10
Train Loss: 1.1254 | Train Acc: 0.6013
Val   Loss: 1.3112 | Val   Acc: 0.4881


In [10]:
resnet50 = get_resnet50_classifier(NUM_CLASSES).to(device)

checkpoint_path = "models/nontransit_resnet50_best_10.pth"
resnet50.load_state_dict(torch.load(checkpoint_path))
optimizer = optim.Adam(resnet50.parameters(), lr=1e-4)

train_config_resume = {
    "model": resnet50,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

model_name = "nontransit_resnet50_best_20"
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train_model(**train_config_resume)
save_model(train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2, model_name)

Epoch 1/10
Train Loss: 1.1132 | Train Acc: 0.5895
Val   Loss: 1.3113 | Val   Acc: 0.4821


Epoch 2/10
Train Loss: 1.1043 | Train Acc: 0.5999
Val   Loss: 1.3186 | Val   Acc: 0.4881


Epoch 3/10
Train Loss: 1.0991 | Train Acc: 0.6013
Val   Loss: 1.3213 | Val   Acc: 0.4821


Epoch 4/10
Train Loss: 1.1003 | Train Acc: 0.6117
Val   Loss: 1.3211 | Val   Acc: 0.4940


Epoch 5/10
Train Loss: 1.1133 | Train Acc: 0.5947
Val   Loss: 1.3104 | Val   Acc: 0.4881


...

Epoch 7/10
Train Loss: 1.1082 | Train Acc: 0.5991
Val   Loss: 1.3115 | Val   Acc: 0.4702


Epoch 8/10
Train Loss: 1.0970 | Train Acc: 0.6102
Val   Loss: 1.3036 | Val   Acc: 0.5000


Epoch 9/10
Train Loss: 1.0876 | Train Acc: 0.6198
Val   Loss: 1.3129 | Val   Acc: 0.4881


Epoch 10/10
Train Loss: 1.0917 | Train Acc: 0.6087
Val   Loss: 1.3109 | Val   Acc: 0.5179
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/liuzehan/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jm/bv3tgs257hx9tx764_p9xq0w0000gn/T/ipykernel_15661/4044437161.py", line 19, in <module>
    save_model(train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2, model_name)
  File "/Users/liuzehan/Desktop/Protected Seas/peter/CNN/engine.py", line 169, in save_model
  File "/Users/liuzehan/miniconda3/lib/python3.11/json/__init__.py", line 179, in dump
    for chunk in iterable:
  File "/Users/liuzehan/miniconda3/lib/python3.11/json/encoder.py", line 432, in _iterencode
    yield from _iterencode_dict(o, _current_indent_level)
  File "/Users/liuzehan/miniconda3/lib/python3.11/json/encoder.py", line 406, in _iterencode_dict
    yield from chunks
  File "/Users/liuzehan/miniconda3/lib/python3.11/json/encoder.py", line 406, in _iterenc